In [ ]:
#importin libraries 
import numpy as np
import os
from os import path
import cv2
from PIL import Image
import keras
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import load_img
rom keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import matplotlib.pyplot as plt




In [ ]:
new_model = tf.keras.models.load_model('data/NN_model.h5')#my model
new_model.summary() #summarising the imported model

In [ ]:
#making the path for the file
dir_path = "data/evaluate"
for i in os.listdir(dir_path):
    #img = image.load_img(dir_path+'//'+ i)
    image1= cv2.imread(dir_path+'//'+ i)
    img= image.load_img(dir_path+'//'+ i, target_size=(256,256))
    img=image.img_to_array(img)
    img=np.array([img])/255

    image = cv2.cvtColor(np.float32(image1),cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower = [np.mean(image[:,:,i] - np.std(image[:,:,i])/3 ) for i in range(3)]
    upper = [250, 250, 250]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask=mask)
    ret, thresh_h = cv2.threshold(mask, 40, 255, 0)
    contours, hierarchy = cv2.findContours(thresh_h, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #find the biggest countour (c) by the area
    c = max(contours, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    # draw the biggest contour (c) in green
    cv2.rectangle(output,(x,y),(x+w,y+h),(0,255,0),5)

    #now we must work on the result:
    result = new_model.predict(img)
    if result <= 0.5:
        print("Tobacco")
        cv2.putText(output, "Tobacco", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 0), 3) 
    else:
        print("Weed")
        cv2.putText(output, "Weed", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 0), 3)

    plt.figure(figsize=(20,4))
    plt.subplot(1,3,1),plt.imshow(image1),plt.title("input")
    plt.subplot(1,3,2),plt.imshow(output),plt.title("Detection")
    plt.show()
